In [ ]:
using FFTW
using Krylov
using LinearAlgebra
using LinearSolve
using LinearMaps

In [ ]:
struct LaplaceFunctor
    Lx:: Float64
    Ly:: Float64
    Nx:: Int
    Ny:: Int
    dx:: Float64
    dy:: Float64
    ncells:: Int
    function LaplaceFunctor(Lx, Ly, Nx, Ny)
        dx = Lx / Nx
        dy = Ly / Ny
        ncells = Nx * Ny
        new(Lx, Ly, Nx, Ny, dx, dy, ncells)
    end
end

function(Δ::LaplaceFunctor)(x)
    x_arr = reshape(x, Δ.Nx, Δ.Ny)
    y = similar(x)
    y_arr = reshape(y, Δ.Nx, Δ.Ny)
    for j₀ ∈ 1:Δ.Ny
        j₋₁ = j₀ == 1 ? Δ.Ny : j₀ - 1
        j₊₁ = j₀ == Δ.Ny ? 1 : j₀ + 1
        for i₀ ∈ 1:Δ.Nx
            i₋₁ = i₀ == 1 ? Δ.Nx : i₀ - 1
            i₊₁ = i₀ == Δ.Nx ? 1 : i₀ + 1
            y_arr[i₀, j₀] = ((x_arr[i₊₁, j₀] - 2 * x_arr[i₀, j₀] + x_arr[i₋₁, j₀]) / Δ.dx^2
                + (x_arr[i₀, j₊₁] - 2 * x_arr[i₀, j₀] + x_arr[i₀, j₋₁]) / Δ.dy^2)
        end
    end
    y
end

function laplace_operator(Lx, Ly, Nx, Ny)
    functor = LaplaceFunctor(Lx, Ly, Nx, Ny)
    LinearMap(functor, functor.ncells, issymmetric=true)
end

In [ ]:
struct LaplaceFunctorFourier
    Lx:: Float64
    Ly:: Float64
    Nx:: Int
    Ny:: Int
    dx:: Float64
    dy:: Float64
    ncells:: Int
    kx²:: Vector{Float64}
    ky²:: Vector{Float64}
    function LaplaceFunctorFourier(Lx, Ly, Nx, Ny)
        dx = Lx / Nx
        dy = Ly / Ny
        ncells = Nx * Ny
        kx² = [(2 * sin(π * n / Nx) / dx)^2 for n = 0:(Nx - 1)]
        ky² = [(2 * sin(π * n / Ny) / dy)^2 for n = 0:(Ny - 1)]
        new(Lx, Ly, Nx, Ny, dx, dy, ncells, kx², ky²)
    end
end

function(Δ:: LaplaceFunctorFourier)(x)
    x_arr = fft(reshape(x, Δ.Nx, Δ.Ny))
    for nx = 1:Δ.Nx
        for ny = 1:Δ.Ny
            x_arr[nx, ny] *= -(Δ.kx²[nx] + Δ.ky²[ny])
        end
    end
    y_arr = real.(ifft(x_arr))
    reshape(y_arr, Δ.ncells)
end

function laplace_operator_fourier(Lx, Ly, Nx, Ny)
    functor = LaplaceFunctorFourier(Lx, Ly, Nx, Ny)
    LinearMap(functor, functor.ncells, issymmetric=true)
end

In [ ]:
Δ = laplace_operator(1.0, 1.5, 64, 128)
Δ_fft = laplace_operator(Δ.f.Lx, Δ.f.Ly, Δ.f.Nx, Δ.f.Ny)


ux = [sin(2π * (n + 0.5) / Δ.f.Nx) for n = 0:(Δ.f.Nx-1)]
uy = [sin(4π * (n + 0.5) / Δ.f.Ny) for n = 0:(Δ.f.Ny-1)]
u = ux .* uy'
u_vec = reshape(u, Δ.f.ncells)

Δu = reshape(Δ * u_vec, Δ.f.Nx, Δ.f.Ny)
Δu_fft = reshape(Δ_fft * u_vec, Δ.f.Nx, Δ.f.Ny)

In [ ]:
@assert all(isapprox.(Δu, Δu_fft, atol=1e-15, rtol=1e-12))

In [ ]:
f_arr = zeros(Float64, Δ.f.Nx, Δ.f.Ny)
f_arr[1, 1] = 1.0
f = reshape(f_arr, Δ.f.ncells)

u0_arr = ones(Float64, Δ.f.Nx, Δ.f.Ny)
u0_arr[1, 1] -= sum(u0_arr)
u0 = reshape(u0_arr, Δ.f.ncells)
(u, stats) = cg(Δ, f, u0)